In [19]:
from puzzle8 import Puzzle8
import math
import random
import time

In [20]:
def neighborhood(puzzle):
    nb = [] # Inicializa uma lista de vizinhos
    blank_row, blank_col = puzzle.get_blank_position() # Retorna a posicao da peça vazia
    # Lista de movimentos cabíveis a uma peça
    moves = [(blank_row - 1, blank_col),  # Cima
             (blank_row + 1, blank_col),  # Baixo
             (blank_row, blank_col - 1),  # Esquerda
             (blank_row, blank_col + 1)]  # Direita

    for move in moves:
        new_row, new_col = move
        if puzzle.is_valid_move(new_row, new_col):  # Verifica se o movimento é válido
            new_puzzle = puzzle.deepcopy()  # Cria uma cópia do tabuleiro atual
            new_puzzle.swap_tiles(blank_row, blank_col, new_row, new_col)  # Troca as posições das peças
            nb.append(new_puzzle)  # Adicionar o novo tabuleiro à lista de vizinhos

    return nb


In [21]:
def hillclimbing(puzzle):
    while True:
        candidates = neighborhood(puzzle) # Obtém a lista de vizinhos possíveis da configuração atual
        better_value = puzzle.evaluate() # Obtém o número de ataques da configuração atual
        better_neighbor = None # Variável para armazenar o melhor vizinho encontrado

        for candidate in candidates:
            value = candidate.evaluate()  # Obtém o número de ataques do vizinho atual
            if( value < better_value ):  # Verifica se o vizinho atual tem menos ataques
                better_neighbor = candidate  # Atualiza o melhor vizinho encontrado
                better_value = value  # Atualiza o número de ataques do melhor vizinho

        if( better_neighbor is None ):  # Se não houver um vizinho melhor
            return puzzle # Retorna a configuração atual (ótimo local)
        else:
            puzzle = better_neighbor # Atualiza a configuração atual para o melhor vizinho encontrado


In [22]:
def hillclimbing_steepest_ascent(puzzle):
    while True:
        current_attacks = puzzle.evaluate() #Obtem o número de desordem da configuração atual
        candidates = neighborhood(puzzle) # Gera candidatos vizinhos
        best_neighbor = None # Inicializa o melhor vizinho
        best_attacks = current_attacks # Inicializar o número mínimo de desordem

        for candidate in candidates:
            candidate_attacks = candidate.evaluate() # Calcula o número de desordem do candidato
            if( candidate_attacks < best_attacks ): # Verificar se o candidato tem menos desordem do que o melhor vizinho atual
                best_neighbor = candidate # Atualiza o melhor vizinho
                best_attacks = candidate_attacks # Atualiza o número mínimo de desordem
        # Verificar se não há melhor vizinho ou se o número mínimo de desordem não diminuiu
        if( ( best_neighbor is None) or (best_attacks >= current_attacks )):
            return puzzle # Retornar a configuração atual
        else:
            puzzle = best_neighbor # Atualizar a configuração atual com o melhor vizinho



In [23]:
def hillclimbing_first_choice(puzzle):
    while True:
        current_attacks = puzzle.evaluate()  # Avalia o número de desordem do estado atual

        best_neighbor = None  # Variável para armazenar o melhor vizinho encontrado

        # Iterar para obter vizinhos aleatórios
        for _ in range(len(puzzle.board)):
            candidate = puzzle.deepcopy()  # Cria uma cópia do tabuleiro atual
            row = random.randint(0, 2)  # Gera um número aleatório para a linha
            col = random.randint(0, 2)  # Gera um número aleatório para a coluna

            # Troca a posição da peça em branco com a posição aleatória gerada
            candidate.swap_tiles(candidate.get_blank_position()[0], candidate.get_blank_position()[1], row, col)

            candidate_attacks = candidate.evaluate()  # Avalia o número de desordem do vizinho

            # Verificar se o vizinho é melhor que o estado atual
            if candidate_attacks < current_attacks:
                best_neighbor = candidate  # Atualiza o melhor vizinho encontrado
                break

        # Se não houver um vizinho melhor, retorna o estado atual
        if best_neighbor is None:
            return puzzle
        else:
            puzzle = best_neighbor  # Atualiza o estado atual para o melhor vizinho encontrado


In [24]:
def hillclimbing_random_restarts(puzzle, max_restarts=25):
    best_solution = None # Inicializa a melhor solução encontrada

    for restart in range(max_restarts):
        current = puzzle.deepcopy() # Cria uma cópia do estado atual
        current = hillclimbing(current) # Aplica o algoritmo Hill Climbing no estado atual
        # Se é a primeira solução ou a solução atual é melhor que a melhor solução anterior
        if( ( best_solution is None) or (current.is_goal_state() < best_solution.is_goal_state() )):
             best_solution = current # Atualiza a melhor solução

        if( best_solution.is_goal_state() == 0 ):
            break # Se encontrou uma solução ideal, interrompe a execução

    return best_solution


In [25]:
def simulated_annealing(puzzle, initial_temperature=20, final_temperature=0.1, alpha=0.5):
    current_state = puzzle  # Estado atual
    current_evaluation = current_state.evaluate()  # Avaliação do estado atual
    best_state = current_state  # Melhor estado encontrado
    best_evaluation = current_evaluation  # Avaliação do melhor estado encontrado
    temperature = initial_temperature  # Temperatura inicial

    while (temperature > final_temperature):
        possible_moves = neighborhood(current_state)  # Obter movimentos possíveis a partir do estado atual
        neighbor = possible_moves[random.randint(0, len(possible_moves) - 1)]  # Escolher um vizinho aleatório
        neighbor_evaluation = neighbor.evaluate()  # Avaliação do vizinho
        evaluation_delta = neighbor_evaluation - current_evaluation  # Diferença de avaliação entre vizinho e estado atual

        # Critério de aceitação do novo estado
        # Representa a probabilidade de aceitar um estado pior (com energia mais alta) como uma forma de escapar de mínimos locais
        if (evaluation_delta < 0) or (random.random() < math.exp(-evaluation_delta / temperature)):
            current_state = neighbor  # Aceitar o vizinho como próximo estado
            current_evaluation = neighbor_evaluation  # Atualizar a avaliação do estado atual

        if (current_evaluation < best_evaluation):
            best_state = current_state  # Atualizar o melhor estado encontrado
            best_evaluation = current_evaluation  # Atualizar a avaliação do melhor estado encontrado

        temperature *= alpha  # Reduzir a temperatura

    return best_state  # Retornar o melhor estado encontrado


In [26]:
def teste(funcao, verbose=False):
    tempo = time.time()
    print(funcao.__name__)
    counter_mean = 0  
    success = 0
    for i in range(maxit):
        counter = 0  
        while True:
            current = Puzzle8() 
            current = funcao(current)  
            counter += 1      
            if( current.is_goal_state()):
                success += 1
                break  
        if( i == maxit-1 and verbose):
            print(current, end="")         
        counter_mean += counter  

    counter_mean /= maxit  
    print("Tentativas: ", counter_mean)   
    print("Sucessos: ", success)
    tempo = time.time() - tempo
    print("Custo: %.2f segundos (%2.2f minutos)\n"%(tempo, tempo/60) )

In [27]:
maxit = 10
teste(hillclimbing)
teste(hillclimbing_steepest_ascent)
teste(hillclimbing_first_choice)
teste(hillclimbing_random_restarts)
teste(simulated_annealing)

hillclimbing
Tentativas:  5397.2
Sucessos:  10
Custo: 16.56 segundos (0.28 minutos)

hillclimbing_steepest_ascent
Tentativas:  5680.0
Sucessos:  10
Custo: 18.57 segundos (0.31 minutos)

hillclimbing_first_choice
Tentativas:  11547.1
Sucessos:  10
Custo: 42.75 segundos (0.71 minutos)

hillclimbing_random_restarts
Tentativas:  3904.5
Sucessos:  10
Custo: 12.83 segundos (0.21 minutos)

simulated_annealing
Tentativas:  55961.9
Sucessos:  10
Custo: 734.59 segundos (12.24 minutos)

